# Unconstrained Optimization
Ricardo A. Fernandes ricardoaf@lccv.ufal.br

Advisor: Professor Adeildo S. Ramos Jr

## Rosenbrock's function

$f\left(\boldsymbol{x}\right) = \left(a - x_1\right)^2 + b\left(x_2 - x_1^2\right)^2$

- Developed by Rosenbrock in 1960
- Well-known unconstrained test function
- Is has a minimum inside a long curved valley
- Some algorithms may have difficulties traversing along the valley to the global minimum
- The global minimum is [$a$, $a^2$], at which $f\left(\boldsymbol{x}^{*}\right) = 0$

In [1]:
rosenbrock(x; a=1, b=5) = (a-x[1])^2 + b*(x[2] - x[1]^2)^2;

In [2]:
using Calculus
∇rosenbrock(x) = Calculus.gradient(rosenbrock, x)
Hrosenbrock(x) = hessian(rosenbrock, x);

## Steepest Descent

In [3]:
using LinearAlgebra

In [4]:
function steepest_descent(f, ∇f, x0; ls=(f,∇f,x,d)->1, tol=1e-6, maxitr=100)
    
    # Init data
    ite = 0; x = x0; conv = false; err = 0
    xh = zeros(length(x0), maxitr+1); xh[:, 1] = x0;
    
    # Iterations
    while ite < maxitr
        
        # search direction
        d = -∇f(x)
        if any(isfinite.(d).==false); break; end
        
        # line search
        α = ls(f, ∇f, x, d)
        
        # update x
        Δx = α * d;
        x += Δx
        
        # update ite and x history
        ite += 1
        xh[:, ite+1] = x
        
        # local minimum point
        err = norm(Δx); if err < tol; conv=true; break; end
    end
    
    # return x history, convergence and error
    return xh[:, 1:ite+1], conv, err
end;

## Conjugate Gradient

In [5]:
function conjugate_gradient(f, ∇f, x0; ls=(f,∇f,x,d)->1, tol=1e-6, maxitr=100)
    
    # Init data
    ite = 0; x = x0; conv = false; err = 0;
    xh = zeros(length(x0), maxitr+1); xh[:, 1] = x0;
    
    # Init conjugate gradient parameters
    g = ∇f(x); d = -g;
    d′, g′ = d, g
    
    # Iterations
    while ite < maxitr
        
        # search direction
        if ite > 0
            g′ = ∇f(x)
            β  = max(0, (g′ ⋅ (g′ - g)) / (g ⋅ g))
            d′ = -g′ + β * d
        end
        if any(isfinite.(d′).==false); break; end
        
        # line search
        α = ls(f, g′, x, d′)
        
        # update x
        Δx = α * d′;
        x += Δx
        
        # update conjugate gradient parameters
        d, g = d′, g′
        
        # update ite and x history
        ite += 1
        xh[:, ite+1] = x
        
        # local minimum point
        err = norm(Δx); if err < tol; conv=true; break; end
    end
    
    # return x history, convergence and error
    return xh[:, 1:ite+1], conv, err
end;

## Newton

In [6]:
function newton(f, ∇f, H, x0; ls=(f,∇f,x,d)->1, tol=1e-6, maxitr=100)
    
    # Init data
    ite = 0; x = x0; conv = false; err = 0
    xh = zeros(length(x0), maxitr+1); xh[:, 1] = x0;
    
    # Iterations
    while ite < maxitr
        
        # search direction
        g = ∇f(x)
        d = -H(x) \ g
        if any(isfinite.(d).==false); break; end
        
        # line search
        α = ls(f, ∇f, x, d)
        
        # update x
        Δx = α * d;
        x += Δx
        
        # update ite and x history
        ite += 1
        xh[:, ite+1] = x
        
        # local minimum point
        err = norm(Δx); if err < tol; conv=true; break; end
    end
    
    # return x history, convergence and error
    return xh[:, 1:ite+1], conv, err
end;

## Evaluating methods

In [7]:
# Initial guess
x0 = [0., 0.]

# post process function
function post_process(name, xh, conv, err)
    println(); println(name)
    println("conv? : ", conv)
    println("# ite : ", size(xh, 2)-1)
    println("  err : ", err)
    println("   x* : ", xh[:, end])
    println("f(x*) : ", rosenbrock(xh[:, end]))
end

# Eval steepest-descent
xh_sd, conv_sd, err_sd = steepest_descent(rosenbrock, ∇rosenbrock, x0)
post_process("steepest-descent", xh_sd, conv_sd, err_sd)

# Eval conjugate-gradient
xh_cg, conv_cg, err_cg = conjugate_gradient(rosenbrock, ∇rosenbrock, x0)
post_process("conjugate-gradient", xh_cg, conv_cg, err_cg)

# Eval newton
xh_n, conv_n, err_n = newton(rosenbrock, ∇rosenbrock, Hrosenbrock, x0)
post_process("newton", xh_n, conv_n, err_n)


steepest-descent
conv? : false
# ite : 6
  err : 3.6025126981219456e230
   x* : [-3.6025126981219456e230, 8.14595958955852e16]
f(x*) : Inf

conjugate-gradient
conv? : false
# ite : 4
  err : 1.917411023934907e165
   x* : [1.9174031098763514e165, 5.508987530057255e162]
f(x*) : Inf

newton
conv? : true
# ite : 4
  err : 3.111490431492185e-11
   x* : [0.9999999996333144, 0.9999999992666289]
f(x*) : 1.3445830783015064e-19


In [8]:
using Plots

In [10]:
plotly(xlabel="x₁", ylabel="x₂", colorbar=false, aspect_ratio=:equal)
lim = (-1.5, 2.5); x = y = LinRange(lim[1], lim[2], 200); 
contour(x, y, (x, y)->rosenbrock([x, y]), levels=100, c=:grey, xlim=lim, ylim=lim)

plot!(xh_sd[1,:], xh_sd[2,:], marker = (:c, 6, 0.6), c=:red, label="steepest-descent")
plot!(xh_cg[1,:], xh_cg[2,:], marker = (:c, 6, 0.6), c=:blue, label="conjugate-gradient")
plot!(xh_n[1,:], xh_n[2,:], marker = (:c, 6, 0.6), c=:black, label="newton")

<!DOCTYPE html>
 
 
 Plots.jl